# Cargar datos y librerias

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Basic
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from statistics import mode
import os
import glob
import random
import cv2

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix
from sklearn.utils import shuffle

import cv2
# Features 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
from skimage import feature
# Metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score #c alculate accuracy
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix
# Classifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import svm
#Save model
import joblib  

# Carga de datos 1

In [ ]:
path = "/content/drive/MyDrive/Universidad - Ingeniería Electrónica/10 Semestre/Trabajo de Grado II/datasets/datasetVGG/4-Asian Indian/train"

list_people = os.listdir(path)

type_file = "*.jpg"

cont = 0
cont2 = 0

for k in list_people:
    
    path_image = path + "/" + k
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    cont += len(list_jpg)
    
print("El número de muestras: ",cont)

train_data = np.zeros((cont,168*192*2))
train_data = train_data.astype('float32')

y_train = np.zeros(cont)
y_train = y_train.astype('int8')


for i in list_people:
    
    #print(i)
    
    path_image = path + "/" + i
    
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    for j in list_jpg:
        
        image = cv2.imread(j, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        image = cv2.resize(image, (168,192))
        H,S,V = cv2.split(image)
        image_merge = np.zeros((192,168,2))
        image_merge[:,:,0] = S
        image_merge[:,:,1] = V
        image_merge = image_merge.ravel()
        train_data[cont2] += image_merge 

        y_train[cont2] = i[-2] + i[-1]

        #print(y_train[cont2])
        cont2 += 1

#print(y_train)

El número de muestras:  2696


In [ ]:
path = "/content/drive/MyDrive/Universidad - Ingeniería Electrónica/10 Semestre/Trabajo de Grado II/datasets/datasetVGG/4-Asian Indian/test"

list_people = os.listdir(path)

subcadena = "yaleB" #la subcadena que queremos localizar
type_file = "*.jpg"

cont = 0
cont2 = 0

for k in list_people:
    
    path_image = path + "/" + k
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    cont += len(list_jpg)
    
print("El número de muestras: ",cont)

test_data = np.zeros((cont,168*192*2))
test_data = test_data.astype('float32')

y_test = np.zeros(cont)
y_test = y_test.astype('int8')


for i in list_people:
    
    #print(i)
    
    path_image = path + "/" + i
    
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    for j in list_jpg:
        
        image = cv2.imread(j, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        image = cv2.resize(image, (168,192))
        H,S,V = cv2.split(image)
        image_merge = np.zeros((192,168,2))
        image_merge[:,:,0] = S
        image_merge[:,:,1] = V
        image_merge = image_merge.ravel()

        test_data[cont2] += image_merge 

        y_test[cont2] = i[-2] + i[-1]
        #print(y_train[cont2])
        cont2 += 1

#print(y_train)

El número de muestras:  1000


#Test

In [ ]:
#image = cv2.imread("/content/drive/MyDrive/Colab Notebooks/Base-De-Datos/datasetVGG/1-African American/train/african_001/0001_02.jpg", cv2.IMREAD_COLOR)

In [ ]:
#image = cv2.resize(image, (168,192))
#H,S,V = cv2.split(image)
#S.shape

In [ ]:
#image_merge = np.zeros((192,168,2))

#image_merge[:,:,0] = S
#image_merge[:,:,1] = V

In [ ]:
#print(V)

In [ ]:
#print(image_merge[:,:,1])

In [ ]:
#train_data, y_train = extract_data(path_train,"*.jpg",True)

In [ ]:
#test_data, y_test = extract_data(path_test,"*.jpg",True)

In [ ]:
#import sys
#np.set_printoptions(threshold=sys.maxsize)
#y_train

# Separar datos en train y test

In [ ]:
#nsamples, nx, ny, chanels = train_data.shape

#train_data = train_data.reshape((nsamples,nx*ny*chanels))

X_sparse = coo_matrix(train_data)

train_data, X_sparse, y_train = shuffle(train_data, X_sparse,y_train, random_state=0)

#train_data = train_data.reshape((nsamples,nx,ny,chanels))

In [ ]:
#nsamples, nx, ny, chanels = test_data.shape

#test_data = test_data.reshape((nsamples,nx*ny*chanels))

X_sparse = coo_matrix(test_data)

test_data, X_sparse, y_test = shuffle(test_data, X_sparse,y_test, random_state=0)

#test_data = test_data.reshape((nsamples,nx,ny,chanels))

In [ ]:
train_data = train_data.astype("uint8")
#nsamples1, nx1, ny1, chanels = train_data.shape
#train_data = train_data.reshape((nsamples1,nx1*ny1*chanels))
train_data = train_data/255

test_data = test_data.astype("uint8")
#nsamples, nx, ny, chanels = test_data.shape
#test_data = test_data.reshape((nsamples,nx*ny*chanels))
test_data = test_data/255

# PCA 

## Extracción de caracteristicas 

In [ ]:
n_components = 150

print("Extrayendo %d eigenfaces de %d caras"
      % (n_components, train_data.shape[0]))

pca = PCA(n_components=n_components, svd_solver='randomized',
          whiten=True).fit(train_data)


#eigenfaces = pca.components_.reshape((n_components,192, 162))

print("Projecting the input data on the eigenfaces orthonormal basis")

train_data_pca = pca.transform(train_data)
test_data_pca = pca.transform(test_data)

Extrayendo 150 eigenfaces de 2696 caras
Projecting the input data on the eigenfaces orthonormal basis


texto en negrita## SVM


## SVM

In [ ]:
from sklearn.model_selection import GridSearchCV


param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }

clf_pca_svm = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)
clf_pca_svm = clf_pca_svm.fit(train_data_pca,y_train )

print("Best estimator found by grid search:")
print(clf_pca_svm.best_estimator_)

Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.005)


In [ ]:
y_pred = clf_pca_svm.predict(test_data_pca)
report = classification_report(y_test, y_pred,digits=4,output_dict=True)
print(classification_report(y_test, y_pred,digits=4))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1     0.9565    0.8800    0.9167        50
           2     0.7447    0.7000    0.7216        50
           3     0.7308    0.7600    0.7451        50
           4     0.9000    0.9000    0.9000        50
           5     0.5556    0.7000    0.6195        50
           6     0.7170    0.7600    0.7379        50
           7     0.8542    0.8200    0.8367        50
           8     0.6731    0.7000    0.6863        50
           9     0.9020    0.9200    0.9109        50
          10     0.6981    0.7400    0.7184        50
          11     0.8654    0.9000    0.8824        50
          12     0.8431    0.8600    0.8515        50
          13     0.7857    0.6600    0.7174        50
          14     0.6226    0.6600    0.6408        50
          15     0.9792    0.9400    0.9592        50
          16     0.6727    0.7400    0.7048        50
          17     0.6379    0.7400    0.6852        50
          18     0.8462    

In [ ]:
import pandas
df = pandas.DataFrame(report).transpose()
df.to_excel("svmPCA.xlsx")  

In [ ]:
def specificity(y_true,y_pred):
        cm=confusion_matrix(y_true, y_pred)
        FP = cm.sum(axis=0) - np.diag(cm)  
        FN = cm.sum(axis=1) - np.diag(cm)
        TP = np.diag(cm)
        TN = cm.sum() - (FP + FN + TP)
        Specificity = TN/(TN+FP)    
        return np.mean(Specificity)
        
print(specificity(y_test, y_pred))

0.9882631578947368


## knn

In [ ]:
clf_PCA_Knn_3 = KNeighborsClassifier(n_neighbors=3)

In [ ]:
clf_lbp_Kn_3 = clf_PCA_Knn_3.fit(train_data_pca,y_train)

In [ ]:
y_pred = clf_PCA_Knn_3.predict(test_data_pca)
report = classification_report(y_test, y_pred,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()                   
df.to_excel("knnPCA.xlsx")
print(classification_report(y_test, y_pred,digits=4))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1     0.5600    0.5600    0.5600        50
           2     0.5091    0.5600    0.5333        50
           3     0.1834    0.6200    0.2831        50
           4     0.8667    0.5200    0.6500        50
           5     0.4118    0.2800    0.3333        50
           6     0.2812    0.5400    0.3699        50
           7     0.3448    0.8000    0.4819        50
           8     0.3750    0.4800    0.4211        50
           9     1.0000    0.4200    0.5915        50
          10     0.3333    0.2200    0.2651        50
          11     0.8800    0.4400    0.5867        50
          12     0.2013    0.6200    0.3039        50
          13     0.4000    0.1200    0.1846        50
          14     0.2963    0.1600    0.2078        50
          15     1.0000    0.3000    0.4615        50
          16     0.5600    0.2800    0.3733        50
          17     0.5600    0.2800    0.3733        50
          18     0.6000    

In [ ]:
from sklearn.metrics import matthews_corrcoef
print(matthews_corrcoef(y_test, y_pred))

0.3661026646796131


In [ ]:
print(specificity(y_test, y_pred))    

0.9678947368421055


## Gauss

In [ ]:
clf_PCA_Gauss = GaussianNB()

In [ ]:
clf_lbp_Gauss = clf_PCA_Gauss.fit(train_data_pca,y_train)

In [ ]:
y_pred = clf_lbp_Gauss.predict(test_data_pca)

report = classification_report(y_test, y_pred,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("gaussPCA.xlsx")
print(classification_report(y_test, y_pred,digits=4))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1     0.8710    0.5400    0.6667        50
           2     0.7143    0.6000    0.6522        50
           3     0.4286    0.6000    0.5000        50
           4     0.7500    0.6600    0.7021        50
           5     0.4364    0.4800    0.4571        50
           6     0.6000    0.6600    0.6286        50
           7     0.6415    0.6800    0.6602        50
           8     0.5405    0.4000    0.4598        50
           9     0.7209    0.6200    0.6667        50
          10     0.5455    0.6000    0.5714        50
          11     0.7736    0.8200    0.7961        50
          12     0.2745    0.8400    0.4138        50
          13     0.5758    0.3800    0.4578        50
          14     0.4167    0.4000    0.4082        50
          15     0.9231    0.7200    0.8090        50
          16     0.4839    0.3000    0.3704        50
          17     0.4364    0.4800    0.4571        50
          18     0.8333    

In [ ]:
print(matthews_corrcoef(y_test, y_pred))

0.545954886784818


In [ ]:
print(specificity(y_test, y_pred)) 

0.9771052631578945


# LDA

## Extracción de caracteristicas 

In [ ]:
lda = LDA(n_components=19)
X_train = lda.fit_transform(train_data, y_train)
train_data_LDA = lda.transform(train_data)
test_data_LDA = lda.transform(test_data)

## SVM

In [ ]:
clf_lda_svm = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)
clf_lda_svm = clf_lda_svm.fit(train_data_LDA,y_train )

print("Best estimator found by grid search:")
print(clf_lda_svm.best_estimator_)

Best estimator found by grid search:
SVC(C=1000.0, class_weight='balanced', gamma=0.01)


In [ ]:
y_pred_lda_svm = clf_lda_svm.predict(test_data_LDA)
report = classification_report(y_test, y_pred_lda_svm,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("svmLDA.xlsx")  
print(classification_report(y_test, y_pred_lda_svm,digits=4))

              precision    recall  f1-score   support

           1     0.9762    0.8200    0.8913        50
           2     0.7391    0.6800    0.7083        50
           3     0.4474    0.6800    0.5397        50
           4     0.9070    0.7800    0.8387        50
           5     0.7500    0.4200    0.5385        50
           6     0.7200    0.7200    0.7200        50
           7     0.4286    0.8400    0.5676        50
           8     0.4079    0.6200    0.4921        50
           9     0.9688    0.6200    0.7561        50
          10     0.7059    0.4800    0.5714        50
          11     0.8409    0.7400    0.7872        50
          12     0.4242    0.8400    0.5638        50
          13     0.7619    0.3200    0.4507        50
          14     0.4925    0.6600    0.5641        50
          15     0.9722    0.7000    0.8140        50
          16     0.5000    0.4000    0.4444        50
          17     0.8387    0.5200    0.6420        50
          18     0.7021    

In [ ]:
print(matthews_corrcoef(y_test, y_pred_lda_svm))

0.6293720205711818


In [ ]:
print(specificity(y_test, y_pred_lda_svm)) 

0.9813157894736839


## knn

In [ ]:
clf_LDA_Knn_3 = KNeighborsClassifier(n_neighbors=3)

In [ ]:
clf_LDA_Knn_3 = clf_PCA_Knn_3.fit(train_data_LDA,y_train)

In [ ]:
y_pred = clf_LDA_Knn_3.predict(test_data_LDA)

report = classification_report(y_test, y_pred,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("knnLDA.xlsx")  
print(classification_report(y_test, y_pred,digits=4))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1     0.9778    0.8800    0.9263        50
           2     0.6491    0.7400    0.6916        50
           3     0.5694    0.8200    0.6721        50
           4     0.8400    0.8400    0.8400        50
           5     0.7045    0.6200    0.6596        50
           6     0.6333    0.7600    0.6909        50
           7     0.8200    0.8200    0.8200        50
           8     0.5455    0.6000    0.5714        50
           9     0.8837    0.7600    0.8172        50
          10     0.5800    0.5800    0.5800        50
          11     0.8478    0.7800    0.8125        50
          12     0.5616    0.8200    0.6667        50
          13     0.7059    0.7200    0.7129        50
          14     0.6522    0.6000    0.6250        50
          15     0.9512    0.7800    0.8571        50
          16     0.6750    0.5400    0.6000        50
          17     0.7143    0.7000    0.7071        50
          18     0.8049    

In [ ]:
print(matthews_corrcoef(y_test, y_pred))

0.7111561550432962


In [ ]:
print(specificity(y_test, y_pred)) 

0.9855263157894736


## Gauss

In [ ]:
clf_LDA_Gauss = GaussianNB()

In [ ]:
clf_LDA_Gauss = clf_LDA_Gauss.fit(train_data_LDA,y_train)

In [ ]:
y_pred = clf_LDA_Gauss.predict(test_data_LDA)

report = classification_report(y_test, y_pred,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("gaussLDA.xlsx") 
print(classification_report(y_test, y_pred,digits=4))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1     0.9744    0.7600    0.8539        50
           2     0.8966    0.5200    0.6582        50
           3     0.5395    0.8200    0.6508        50
           4     1.0000    0.7000    0.8235        50
           5     0.9000    0.3600    0.5143        50
           6     0.8421    0.6400    0.7273        50
           7     0.3382    0.9200    0.4946        50
           8     0.5926    0.6400    0.6154        50
           9     1.0000    0.4200    0.5915        50
          10     0.8824    0.3000    0.4478        50
          11     0.8750    0.7000    0.7778        50
          12     0.2541    0.9200    0.3983        50
          13     0.8824    0.3000    0.4478        50
          14     0.3871    0.7200    0.5035        50
          15     1.0000    0.5800    0.7342        50
          16     0.9048    0.3800    0.5352        50
          17     0.9474    0.3600    0.5217        50
          18     0.9062    

In [ ]:
print(matthews_corrcoef(y_test, y_pred))

0.5942195115708553


In [ ]:
print(specificity(y_test, y_pred)) 

0.9791578947368421


# Carga de datos 2

In [ ]:
path = "/content/drive/MyDrive/Universidad - Ingeniería Electrónica/10 Semestre/Trabajo de Grado II/datasets/datasetVGG/4-Asian Indian/train"

list_people = os.listdir(path)

type_file = "*.jpg"

cont = 0
cont2 = 0

for k in list_people:
    
    path_image = path + "/" + k
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    cont += len(list_jpg)
    
print("El número de muestras: ",cont)

train_data = np.zeros((cont,192,168,2))
train_data = train_data.astype('float32')

y_train = np.zeros(cont)
y_train = y_train.astype('int8')


for i in list_people:
    
    #print(i)
    
    path_image = path + "/" + i
    
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    for j in list_jpg:
        
        image = cv2.imread(j, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        image = cv2.resize(image, (168,192))
        H,S,V = cv2.split(image)
        image_merge = np.zeros((192,168,2))
        image_merge[:,:,0] = S
        image_merge[:,:,1] = V
        train_data[cont2] += image_merge 

        y_train[cont2] = i[-2] + i[-1]

        #print(y_train[cont2])
        cont2 += 1

#print(y_train)

El número de muestras:  2696


In [ ]:
path = "/content/drive/MyDrive/Universidad - Ingeniería Electrónica/10 Semestre/Trabajo de Grado II/datasets/datasetVGG/4-Asian Indian/test"

list_people = os.listdir(path)

subcadena = "yaleB" #la subcadena que queremos localizar
type_file = "*.jpg"

cont = 0
cont2 = 0

for k in list_people:
    
    path_image = path + "/" + k
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    cont += len(list_jpg)
    
print("El número de muestras: ",cont)

test_data = np.zeros((cont,192,168,2))
test_data = test_data.astype('float32')

y_test = np.zeros(cont)
y_test = y_test.astype('int8')


for i in list_people:
    
    #print(i)
    
    path_image = path + "/" + i
    
    list_jpg = glob.glob(path_image + os.sep + type_file)
    
    for j in list_jpg:
        
        image = cv2.imread(j, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        image = cv2.resize(image, (168,192))
        H,S,V = cv2.split(image)
        image_merge = np.zeros((192,168,2))
        image_merge[:,:,0] = S
        image_merge[:,:,1] = V
        

        test_data[cont2] += image_merge 

        y_test[cont2] = i[-2] + i[-1]
        #print(y_train[cont2])
        cont2 += 1

#print(y_train)

El número de muestras:  1000


# Separar datos en train y test

In [ ]:
nsamples, nx, ny, chanels = train_data.shape

train_data = train_data.reshape((nsamples,nx*ny*chanels))

X_sparse = coo_matrix(train_data)

train_data, X_sparse, y_train = shuffle(train_data, X_sparse,y_train, random_state=0)

train_data = train_data.reshape((nsamples,nx,ny,chanels))

In [ ]:
nsamples, nx, ny, chanels = test_data.shape

test_data = test_data.reshape((nsamples,nx*ny*chanels))

X_sparse = coo_matrix(test_data)

test_data, X_sparse, y_test = shuffle(test_data, X_sparse,y_test, random_state=0)

test_data = test_data.reshape((nsamples,nx,ny,chanels))

In [ ]:
train_data = train_data.astype("uint8")
nsamples1, nx1, ny1, chanels = train_data.shape
train_data = train_data.reshape((nsamples1,nx1*ny1*chanels))
train_data = train_data/255

test_data = test_data.astype("uint8")
nsamples, nx, ny, chanels = test_data.shape
test_data = test_data.reshape((nsamples,nx*ny*chanels))
test_data = test_data/255

In [ ]:
train_data = train_data.reshape((nsamples1,nx1,ny1,chanels))
test_data = test_data.reshape((nsamples,nx,ny,chanels))

In [ ]:
train_data_S = np.zeros((nsamples1,192,168))
train_data_S = train_data[:,:,:,0]
train_data_V = np.zeros((nsamples1,192,168))
train_data_V = train_data[:,:,:,1]

test_data_S = np.zeros((nsamples,192,168))
test_data_S = test_data[:,:,:,0]
test_data_V = np.zeros((nsamples1,192,168))
test_data_V = test_data[:,:,:,1]

# LBP


## Extracción de caracteristicas

In [ ]:
# https://github.com/AsoStrife/Computer-Vision-Project

def blockshaped(arr, nrows, ncols):

    h, w = arr.shape
    return (arr.reshape(h//nrows, nrows, -1, ncols)
            .swapaxes(1,2)
            .reshape(-1, nrows, ncols))
def getHistogram(imgArray):
    hist, bin_edges = np.histogram(imgArray, density=True)
    return hist

In [ ]:
# tamaño de train y test
cont_train = train_data.shape[0]
cont_test = test_data.shape[0]
print(cont_test)

1000


In [ ]:
#train_data_LBP = train_data.reshape((cont_train,nx,ny))
lbp_feactures  = np.zeros((cont_train,nx,ny))

eps = 1e-7
radius = 1
n_points = 8 * radius

lbp_hist_S = np.zeros((cont_train,1440))

for i in range(0,cont_train): 
    lbp = feature.local_binary_pattern(train_data_S[i], n_points, radius)
    lbp_feactures[i] +=  lbp
    
    shaped = blockshaped(lbp_feactures[i], 16, 14)
    x = []
    xBlocks = []
    for s in shaped:
        xBlocks.append(getHistogram(s))
    # Concatenate the various histogram, the resulting histogram is append into feature vector
    x.append(np.concatenate(xBlocks))
    
    lbp_hist_S[i] += x[0]

In [ ]:
#train_data_LBP = train_data.reshape((cont_train,nx,ny))
lbp_feactures  = np.zeros((cont_train,nx,ny))

eps = 1e-7
radius = 1
n_points = 8 * radius

lbp_hist_V = np.zeros((cont_train,1440))

for i in range(0,cont_train): 
    lbp = feature.local_binary_pattern(train_data_V[i], n_points, radius)
    lbp_feactures[i] +=  lbp
    
    shaped = blockshaped(lbp_feactures[i], 16, 14)
    x = []
    xBlocks = []
    for s in shaped:
        xBlocks.append(getHistogram(s))
    # Concatenate the various histogram, the resulting histogram is append into feature vector
    x.append(np.concatenate(xBlocks))
    
    lbp_hist_V[i] += x[0]

In [ ]:
#test_data_LBP = test_data.reshape((cont_test,nx,ny))
lbp_feactures_test  = np.zeros((cont_test,nx,ny))


lbp_hist_test_S = np.zeros((cont_test,1440))

for i in range(0,cont_test): 
    lbp = feature.local_binary_pattern(test_data_S[i], n_points, radius)
    lbp_feactures_test[i] +=  lbp
    
    shaped = blockshaped(lbp_feactures_test[i], 16, 14)
    x = []
    xBlocks = []
    for s in shaped:
        xBlocks.append(getHistogram(s))
    # Concatenate the various histogram, the resulting histogram is append into feature vector
    x.append(np.concatenate(xBlocks))
    
    lbp_hist_test_S[i] += x[0]
    

In [ ]:
#test_data_LBP = test_data.reshape((cont_test,nx,ny))
lbp_feactures_test  = np.zeros((cont_test,nx,ny))


lbp_hist_test_V = np.zeros((cont_test,1440))

for i in range(0,cont_test): 
    lbp = feature.local_binary_pattern(test_data_V[i], n_points, radius)
    lbp_feactures_test[i] +=  lbp
    
    shaped = blockshaped(lbp_feactures_test[i], 16, 14)
    x = []
    xBlocks = []
    for s in shaped:
        xBlocks.append(getHistogram(s))
    # Concatenate the various histogram, the resulting histogram is append into feature vector
    x.append(np.concatenate(xBlocks))
    
    lbp_hist_test_V[i] += x[0]
    

In [ ]:
lbp_hist_test_V.shape

(1000, 1440)

In [ ]:
lbp_hist = np.concatenate((lbp_hist_S,lbp_hist_V),axis=1)
lbp_hist.shape

(2696, 2880)

In [ ]:
lbp_hist_test = np.concatenate((lbp_hist_test_S,lbp_hist_test_V),axis=1)
lbp_hist_test.shape

(1000, 2880)

## SMV

In [ ]:
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf_lbp_svm = GridSearchCV(
    SVC(kernel='rbf', class_weight='balanced'), param_grid
)
clf_lbp_svm = clf_lbp_svm.fit(lbp_hist,y_train )

print("El mejor conjunto de caracteriticas es: ")
print(clf_lbp_svm.best_estimator_)

El mejor conjunto de caracteriticas es: 
SVC(C=50000.0, class_weight='balanced', gamma=0.001)


In [ ]:
y_pred_lbp_svm = clf_lbp_svm.predict(lbp_hist_test)

report = classification_report(y_test, y_pred_lbp_svm,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("svmLBP.xlsx")
print(classification_report(y_test, y_pred_lbp_svm,digits=4))

              precision    recall  f1-score   support

           1     0.9302    0.8000    0.8602        50
           2     0.7917    0.7600    0.7755        50
           3     0.7636    0.8400    0.8000        50
           4     0.8571    0.9600    0.9057        50
           5     0.8542    0.8200    0.8367        50
           6     0.6557    0.8000    0.7207        50
           7     0.7885    0.8200    0.8039        50
           8     0.7955    0.7000    0.7447        50
           9     0.8846    0.9200    0.9020        50
          10     0.8000    0.8000    0.8000        50
          11     0.8571    0.8400    0.8485        50
          12     0.7736    0.8200    0.7961        50
          13     0.7755    0.7600    0.7677        50
          14     0.7115    0.7400    0.7255        50
          15     0.9375    0.9000    0.9184        50
          16     0.7358    0.7800    0.7573        50
          17     0.7778    0.8400    0.8077        50
          18     0.8571    

In [ ]:
print(matthews_corrcoef(y_test, y_pred_lbp_svm))

0.8033456458696784


In [ ]:
print(specificity(y_test, y_pred_lbp_svm)) 

0.9901578947368421


## KNN

In [ ]:
clf_lbp_Kn_3 = KNeighborsClassifier(n_neighbors=7)

In [ ]:
clf_lbp_Kn_3 = clf_lbp_Kn_3.fit(lbp_hist,y_train)

In [ ]:
y_pred_lbp_knn = clf_lbp_Kn_3.predict(lbp_hist_test)

report = classification_report(y_test, y_pred_lbp_knn,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()                   
df.to_excel("knnLBP.xlsx")  
print(classification_report(y_test, y_pred_lbp_knn,digits=4))

              precision    recall  f1-score   support

           1     0.3860    0.8800    0.5366        50
           2     0.4681    0.4400    0.4536        50
           3     0.3117    0.4800    0.3780        50
           4     0.7547    0.8000    0.7767        50
           5     0.6000    0.6000    0.6000        50
           6     0.4444    0.4800    0.4615        50
           7     0.6250    0.4000    0.4878        50
           8     0.5610    0.4600    0.5055        50
           9     0.6600    0.6600    0.6600        50
          10     0.4894    0.4600    0.4742        50
          11     0.8148    0.4400    0.5714        50
          12     0.6087    0.2800    0.3836        50
          13     0.3171    0.5200    0.3939        50
          14     0.3750    0.1800    0.2432        50
          15     0.8889    0.6400    0.7442        50
          16     0.4603    0.5800    0.5133        50
          17     0.3077    0.6400    0.4156        50
          18     0.5417    

In [ ]:
print(matthews_corrcoef(y_test, y_pred_lbp_knn))

0.4612034822508878


In [ ]:
print(specificity(y_test, y_pred_lbp_knn))  

0.9728947368421054


## Gauss

In [ ]:
clf_lbp_Gauss = GaussianNB()

In [ ]:
clf_lbp_Gauss = clf_lbp_Gauss.fit(lbp_hist,y_train)

In [ ]:
y_pred_lbp_gaus = clf_lbp_Gauss.predict(lbp_hist_test)

report = classification_report(y_test, y_pred_lbp_gaus,digits=4,output_dict=True)
df = pandas.DataFrame(report).transpose()
df.to_excel("gaussLBP.xlsx")
print(classification_report(y_test, y_pred_lbp_gaus,digits=4))

              precision    recall  f1-score   support

           1     0.5484    0.3400    0.4198        50
           2     0.4737    0.3600    0.4091        50
           3     0.4000    0.3600    0.3789        50
           4     0.6341    0.5200    0.5714        50
           5     0.3297    0.6000    0.4255        50
           6     0.3333    0.2800    0.3043        50
           7     0.5714    0.4800    0.5217        50
           8     0.4783    0.2200    0.3014        50
           9     0.6122    0.6000    0.6061        50
          10     0.3182    0.1400    0.1944        50
          11     0.5000    0.5000    0.5000        50
          12     0.2899    0.4000    0.3361        50
          13     0.3898    0.4600    0.4220        50
          14     0.2321    0.2600    0.2453        50
          15     0.7692    0.6000    0.6742        50
          16     0.2941    0.2000    0.2381        50
          17     0.2000    0.6600    0.3070        50
          18     0.4118    

In [ ]:
print(matthews_corrcoef(y_test, y_pred_lbp_gaus))

0.3614743143559303


In [ ]:
print(specificity(y_test, y_pred_lbp_gaus)) 

0.9678947368421053
